<a href="https://colab.research.google.com/github/edtorrente/big-data-challenge/blob/main/Level1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-25 17:59:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.56MB/s    in 0.2s    

2022-02-25 17:59:35 (5.56 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Video").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_v1_00.tsv.gz"), inferSchema=True, sep='\t', header=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49033728|R1P1G5KZ05H6RD|6302503213|     748506413|The Night They Sa...|           Video|          5|            0|          0|   N|                Y|    Very satisfied!!|Fast shipping. Pl...| 2015-08-31|
|         US|   17857748|R106N066IUN8ZV|B000059PET|     478710180|Hamlet / Kline, N...|           Video|          5|    

In [ ]:
print(df.count())

4850360


In [6]:
df=df.dropna()
print(df.count())

df1=df.dropDuplicates()
print(df1.count())

380575
380575


In [7]:
df1.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



#Transform dataframe, df to fit review_id schema sql

In [8]:
review_id=df1.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1000G235G84VS|   39107297|6304428456|     619227014| 2002-07-30|
|R103NDNKDVKGYD|   46346809|B00AVPYKFU|     631259326| 2015-04-24|
|R1068F9HHXTYR5|   42564085|6304898592|     478267204| 2004-09-17|
|R10D7QYJVUDBJI|   46499989|1574924494|     716139900| 2001-01-02|
|R10HY4HIY7HLIY|   50133272|6301980506|     557595408| 2009-11-15|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [9]:
review_id=review_id.dropDuplicates(["review_id"])

## Typecast selected columns to match in provided sql schema


In [10]:
from pyspark.sql.types import * 

review_id=review_id.withColumn("review_date", review_id["review_date"].cast(DateType()))

review_id.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



#

#Transform dataframe, df to fit product_id schema sql

In [11]:
products_df=df1.select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|6304428456|      Daylight [VHS]|
|B00AVPYKFU|Blank VHS Tape So...|
|6304898592|Baby's First Impr...|
|1574924494|   The Odyssey [VHS]|
|6301980506|Strike Up the Ban...|
+----------+--------------------+
only showing top 5 rows



In [12]:
print(products_df.count())

380575


In [13]:
products_df=products_df.dropDuplicates(["product_id"])

In [14]:
print(products_df.count())

77020


In [15]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



#Transform customer_id table

In [16]:
customers_df =df1.select(["customer_id"])
customers_df.show(5)

+-----------+
|customer_id|
+-----------+
|   39107297|
|   46346809|
|   42564085|
|   46499989|
|   50133272|
+-----------+
only showing top 5 rows



In [17]:
customers_df=customers_df.groupBy("customer_id").count()
customers_df.orderBy("customer_id").select(["customer_id", "count"])
customers_df.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   20288432|    2|
|   49145342|    2|
|   21187650|    2|
|   27170594|    1|
|   36395157|    5|
+-----------+-----+
only showing top 5 rows



In [18]:
customers_df=customers_df.withColumnRenamed("count", "customer_count")
customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   20288432|             2|
|   49145342|             2|
|   21187650|             2|
|   27170594|             1|
|   36395157|             5|
+-----------+--------------+
only showing top 5 rows



In [19]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [20]:
from pyspark.sql.types import * 



customers_df=customers_df.withColumn("customer_count", customers_df["customer_count"].cast(IntegerType()))

customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [21]:
# select columns to fit the vine table schema

vine_df=df1.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1000G235G84VS|          5|            0|          7|   N|
|R103NDNKDVKGYD|          5|            0|          0|   N|
|R1068F9HHXTYR5|          1|            0|          0|   N|
|R10D7QYJVUDBJI|          3|            0|          1|   N|
|R10HY4HIY7HLIY|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [22]:
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [23]:
vine_df=vine_df.dropDuplicates(["review_id"])

#Write DataFrame to RDS

In [24]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<input endpoint>:5432/<database name>"
config = {"user":"<input username>",
          "password": "<input password>",
          "driver":"org.postgresql.Driver"}

In [25]:
review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [26]:
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [27]:
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [28]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)